In [1]:
!pip install pyarrow

In [2]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

In [3]:
dfJanuary = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
dfFebruary = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [4]:
dfJanuary.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
dfFebruary.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


### Q1: Read the data for January. How many columns are there?

In [6]:
dfJanuary.shape[1]

19

### Q2: What's the standard deviation of the trips duration in January?

In [7]:
# Create two new columns that contain the amount of minutes of the trip duration. This info is in tpep_pickup_datetime and tpep_dropoff_datetime columns.
dfJanuary['duration'] = dfJanuary.tpep_dropoff_datetime - dfJanuary.tpep_pickup_datetime

dfJanuary.duration = dfJanuary.duration.apply(lambda x: x.total_seconds() / 60)

In [8]:
dfJanuary.duration.std()

42.594351241920904

### Q3: What fraction of the records left after you dropped the outliers?

In [9]:
total = dfJanuary[(dfJanuary['duration'] > 60) | (dfJanuary['duration'] < 1)].shape[0]

dfJanuary = dfJanuary[(dfJanuary['duration'] >= 0) & (dfJanuary['duration'] <= 60)]
filtered = dfJanuary.shape[0]

# Calculate the percentage of trips that were removed.
percentage = 100 - (total / (total + filtered)) * 100
percentage

98.1422200201413

### Q4: One-hot encoding. What's the dimensionality of this matrix (number of columns)?

In [10]:
# Apply one-hot encoding to PUlocationID and DOlocationID columns.
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dfJanuary['DOLocationID'] = dfJanuary['DOLocationID'].astype(str)
dfJanuary['PULocationID'] = dfJanuary['PULocationID'].astype(str)

In [11]:
train_dicts = dfJanuary[categorical + numerical].to_dict(orient='records')

In [12]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [13]:
X_train.shape


(3042505, 518)

### Q5: Training a model

What is the RMSE on train?

In [14]:
y_train = dfJanuary['duration'].values
y_train

array([ 8.43333333,  6.31666667, 12.75      , ..., 24.51666667,
       13.        , 14.4       ])

In [15]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [16]:
# Calculate RMSE on training data.
from sklearn.metrics import mean_squared_error

y_pred = lr.predict(X_train)
rmse = mean_squared_error(y_train, y_pred, squared=False)
rmse

7.9285377849545835

### Q5: Training a model

What is the RMSE on validation?

In [17]:
dfFebruary['duration'] = dfFebruary.tpep_dropoff_datetime - dfFebruary.tpep_pickup_datetime
dfFebruary.duration = dfFebruary.duration.apply(lambda x: x.total_seconds() / 60)

dfFebruary = dfFebruary[(dfFebruary['duration'] >= 0) & (dfFebruary['duration'] <= 60)]

dfFebruary['DOLocationID'] = dfFebruary['DOLocationID'].astype(str)
dfFebruary['PULocationID'] = dfFebruary['PULocationID'].astype(str)

In [18]:
test_dicts = dfFebruary[categorical + numerical].to_dict(orient='records')
X_test = dv.transform(test_dicts)

In [19]:
y_test = dfFebruary['duration'].values
y_pred = lr.predict(X_test)

In [25]:
rmse2 = mean_squared_error(y_test, y_pred, squared=False)
rmse2

8.09956780418923